# CSV + API

In this reboot, we are going to use:

- The [Goodreads books](https://www.kaggle.com/jealousleopard/goodreadsbooks) dataset from Kaggle.
- The [Open Library Books API](https://openlibrary.org/dev/docs/api/books)

The goal of this livecode is to load the data from a CSV + loop over rows to enrich each row with information such as:

- List of subjects (Science, Humor, Travel, etc.)
- The cover URL of the book
- Other information you'd find useful in the JSON API

First, download the CSV in the local folder:

In [1]:
#!curl -L https://gist.githubusercontent.com/ssaunier/351b17f5a7a009808b60aeacd1f4a036/raw/books.csv > books.csv

Then import the usual suspects!

In [2]:
import pandas as pd
import requests
import numpy as np

In [3]:
books_df = pd.read_csv('books.csv')

In [4]:
books_df.drop(columns=['bookID', 'text_reviews_count', 'ratings_count', 'isbn'], inplace=True)

In [5]:
books_df.head(3)

,title,authors,average_rating,isbn13,language_code,# num_pages
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,9780439785969,eng,652
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,9780439358071,eng,870
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,9780439554930,eng,320


In [6]:
'https://openlibrary.org/api/books?bibkeys=ISBN:9780439785969&format=json&jscmd=data'

'https://openlibrary.org/api/books?bibkeys=ISBN:9780439785969&format=json&jscmd=data'

In [7]:
books_df['isbn13'][0]

9780439785969

In [8]:
url = 'https://openlibrary.org/api/books'

isbn13 = books_df['isbn13'][0]

params = {
    'bibkeys': f'ISBN:{isbn13}',
    'format': 'json',
    'jscmd': 'data'
}

requests.get(url, params).json()

{'ISBN:9780439785969': {'url': 'https://openlibrary.org/books/OL24280830M/Harry_Potter_and_the_Half-Blood_Prince',
  'key': '/books/OL24280830M',
  'title': 'Harry Potter and the Half-Blood Prince',
  'authors': [{'url': 'https://openlibrary.org/authors/OL23919A/J._K._Rowling',
    'name': 'J. K. Rowling'}],
  'identifiers': {'amazon': ['0439785960'],
   'goodreads': ['53178655'],
   'isbn_10': ['0439785960'],
   'isbn_13': ['9780439785969'],
   'oclc': ['70666878', '819153929'],
   'openlibrary': ['OL24280830M']},
  'publishers': [{'name': 'Scholastic'}],
  'publish_places': [{'name': 'New York, USA'}],
  'publish_date': '2006-09',
  'subjects': [{'name': 'orphans',
    'url': 'https://openlibrary.org/subjects/orphans'},
   {'name': 'foster homes',
    'url': 'https://openlibrary.org/subjects/foster_homes'},
   {'name': 'romans', 'url': 'https://openlibrary.org/subjects/romans'},
   {'name': 'magie', 'url': 'https://openlibrary.org/subjects/magie'},
   {'name': 'adolescence',
    'url

In [18]:
def fetch_cover_url(isbn13):
    url = 'https://openlibrary.org/api/books'
    
    params = {
    'bibkeys': f'ISBN:{isbn13}',
    'format': 'json',
    'jscmd': 'data'
    }

    response = requests.get(url, params).json()
    if f'ISBN:{isbn13}' in response:
        return response[f"ISBN:{isbn13}"].get("cover", {}).get("large", "")
    return ''

In [23]:
fetch_cover_url('9780439785969')

'https://covers.openlibrary.org/b/id/9326654-L.jpg'

In [24]:
books_df['cover_url'] = None

In [25]:
%%time

# TODO: for row in rows => fetch_book => add column

for index, row in books_df.head(15).iterrows():
    if row["cover_url"] is None:
        isbn = row['isbn13']
        print(f"Fetching cover for {row['title']}")
        cover_url = fetch_cover_url(isbn)
        if cover_url:
            books_df.loc[index, "cover_url"] = cover_url
        else:
            books_df.loc[index, "cover_url"] = ""

Fetching cover for Harry Potter and the Half-Blood Prince (Harry Potter  #6)
Fetching cover for Harry Potter and the Order of the Phoenix (Harry Potter  #5)
Fetching cover for Harry Potter and the Sorcerer's Stone (Harry Potter  #1)
Fetching cover for Harry Potter and the Chamber of Secrets (Harry Potter  #2)
Fetching cover for Harry Potter and the Prisoner of Azkaban (Harry Potter  #3)
Fetching cover for Harry Potter Boxed Set  Books 1-5 (Harry Potter  #1-5)
Fetching cover for Unauthorized Harry Potter Book Seven News: "Half-Blood Prince" Analysis and Speculation
Fetching cover for Harry Potter Collection (Harry Potter  #1-6)
Fetching cover for The Ultimate Hitchhiker's Guide: Five Complete Novels and One Story (Hitchhiker's Guide to the Galaxy  #1-5)
Fetching cover for The Ultimate Hitchhiker's Guide to the Galaxy
Fetching cover for The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy  #1)
Fetching cover for The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide t

In [26]:
books_df.head(15)

,title,authors,average_rating,isbn13,language_code,# num_pages,cover_url
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,9780439785969,eng,652,https://covers.openlibrary.org/b/id/9326654-L.jpg
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,9780439358071,eng,870,https://covers.openlibrary.org/b/id/12025650-L...
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,9780439554930,eng,320,https://covers.openlibrary.org/b/id/7572543-L.jpg
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,9780439554893,eng,352,https://covers.openlibrary.org/b/id/10301720-L...
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,9780439655484,eng,435,https://covers.openlibrary.org/b/id/10580458-L...
5,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,4.78,9780439682589,eng,2690,https://covers.openlibrary.org/b/id/278981-L.jpg
6,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.69,9780976540601,en-US,152,https://covers.openlibrary.org/b/id/742235-L.jpg
7,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,9780439827607,eng,3342,https://covers.openlibrary.org/b/id/279436-L.jpg
8,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,9780517226957,eng,815,https://covers.openlibrary.org/b/id/12617870-L...
9,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,4.38,9780345453747,eng,815,


In [ ]:
books_df['cover_url'] = None

In [22]:
%%time
books_df['cover_url'] = books_df.head(15).apply(lambda row: fetch_cover_url(row['isbn13']), axis=1)

CPU times: user 300 ms, sys: 14.3 ms, total: 314 ms
Wall time: 20.9 s


In [20]:
books_df.head(15)

,title,authors,average_rating,isbn13,language_code,# num_pages,cover_url
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,9780439785969,eng,652,https://covers.openlibrary.org/b/id/9326654-L.jpg
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,9780439358071,eng,870,https://covers.openlibrary.org/b/id/12025650-L...
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,9780439554930,eng,320,https://covers.openlibrary.org/b/id/7572543-L.jpg
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,9780439554893,eng,352,https://covers.openlibrary.org/b/id/10301720-L...
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,9780439655484,eng,435,https://covers.openlibrary.org/b/id/10580458-L...
5,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,4.78,9780439682589,eng,2690,https://covers.openlibrary.org/b/id/278981-L.jpg
6,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.69,9780976540601,en-US,152,https://covers.openlibrary.org/b/id/742235-L.jpg
7,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,9780439827607,eng,3342,https://covers.openlibrary.org/b/id/279436-L.jpg
8,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,9780517226957,eng,815,https://covers.openlibrary.org/b/id/12617870-L...
9,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,4.38,9780345453747,eng,815,


## Calling the API with multiple ISBNs at a time

In [27]:
isbns = [9780439785969, 9780439358071, 9780439554930]
[f"ISBN:{isbn}" for isbn in isbns]

['ISBN:9780439785969', 'ISBN:9780439358071', 'ISBN:9780439554930']

In [28]:
",".join([f"ISBN:{isbn}" for isbn in isbns])

'ISBN:9780439785969,ISBN:9780439358071,ISBN:9780439554930'

In [29]:
def fetch_books(isbns):
    url = "https://openlibrary.org/api/books"
    bibkeys = ",".join([f"ISBN:{isbn}" for isbn in isbns])
    params = {
        'bibkeys': bibkeys,
        'format': 'json',
        'jscmd': 'data'
    }
    response = requests.get(url, params=params).json()
    return response

In [41]:
np.array_split(books_df[['title']].head(20), 5)[4]

,title
16,In a Sunburned Country
17,I'm a Stranger Here Myself: Notes on Returning...
18,The Lost Continent: Travels in Small Town America
19,Neither Here nor There: Travels in Europe


In [44]:
books_df.set_index("isbn13", inplace=True)

In [47]:
%%time

for group in np.array_split(books_df.head(100), 5): # 5 groups of 20 books
    books = fetch_books(list(group.index))
    for isbn_code, book in books.items():
        isbn = int(isbn_code.strip("ISBN:"))
        books_df.loc[isbn, "cover_url"] = book.get("cover", {}).get("large", "")

CPU times: user 149 ms, sys: 34.1 ms, total: 183 ms
Wall time: 16.2 s


In [48]:
books_df.head(100)

,title,authors,average_rating,language_code,# num_pages,cover_url
isbn13,,,,,,
9780439785969,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,eng,652,https://covers.openlibrary.org/b/id/9326654-L.jpg
9780439358071,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,eng,870,https://covers.openlibrary.org/b/id/12025650-L...
9780439554930,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,eng,320,https://covers.openlibrary.org/b/id/7572543-L.jpg
9780439554893,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,eng,352,https://covers.openlibrary.org/b/id/10301720-L...
9780439655484,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,eng,435,https://covers.openlibrary.org/b/id/10580458-L...
...,...,...,...,...,...,...
9780451528612,Anna Karenina,Leo Tolstoy-David Magarshack-Priscilla Meyer,4.04,eng,960,https://covers.openlibrary.org/b/id/295745-L.jpg
9780140449174,Anna Karenina,Leo Tolstoy-Richard Pevear-Larissa Volokhonsky...,4.04,eng,837,
9780822001836,CliffsNotes on Tolstoy's Anna Karenina,Marianne Sturman-Leo Tolstoy,3.89,eng,80,https://covers.openlibrary.org/b/id/6951821-L.jpg
